In [8]:
import numpy as np
from mnist import load_mnist

In [9]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

train_loss_list = []

hyperparameter

In [10]:
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100 #minibatch size
learning_rate = 0.1

neural network and necessary functions

In [23]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    c = np.max(x)
    exp_x = np.exp(x-c)
    sum_exp_x = np.sum(exp_x)
    y = exp_x / sum_exp_x
    
    return y

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y + 1e-7)) / batch_size    

def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    
    if x.ndim == 1: #in case x is (n,).
        for i in range(x.size):
            tmp_val = x[i]
            x[i] = tmp_val + h
            fxh1 = f(x)

            x[i] = tmp_val - h
            fxh2 = f(x)
    
            grad[i] = (fxh1 - fxh2) / (2*h)
            x[i] = tmp_val
        
        return grad
    
    else:
        for i in range(len(x)):
            for j in range(len(x[0])):
                tmp_val = x[i][j]
                x[i][j] = tmp_val + h
                fxh1 = f(x)
    
                x[i][j] = tmp_val - h
                fxh2 = f(x)
    
                grad[i][j] = (fxh1 - fxh2) / (2*h)
                x[i][j] = tmp_val
    
        return grad

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        
        self.params={}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x,W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1,W2) + b2
        z2 = sigmoid(a2)
        y = softmax(z2)
        
        return y
    
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def numerical_gradient(self, x, t): #calculate the gradient value for W1, b1, W2, b2 respectively.
        def loss_W(W):
            return self.loss(x,t) #loss_W = lambda W: self.loss(x,t) instead.
        
        grads={}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [26]:
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

training with minibatch

In [ ]:
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size) # choose batch_size samples out of train_size datas
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.numerical_gradient(x_batch, t_batch)
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)